In [2]:
### '''匯入套件'''
import json, os, pprint, time, re
import requests
from bs4 import BeautifulSoup

'''放置  metadata 的資訊'''
listData = []

'''古騰堡網址'''
domain = 'https://www.gutenberg.org/'
url = 'https://www.gutenberg.org/browse/languages/zh'

'''設定標頭'''
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
}

# 取得主要連結
def getLinks():
    response = requests.get(url, headers = headers,)
    soup = BeautifulSoup(response.text, "lxml")
    a_elms = soup.select('li.pgdbetext a',limit=250)
    
    for a in a_elms: 
        listData.append({
        "title": a.text,
        "link": domain + a.get('href')
        })
        
        
def subLinks():
    for i in range(250):
        listData[i]['sub']=[]
        response = requests.get(listData[i]['link'], headers = headers)
        soup = BeautifulSoup(response.text, "lxml")
        a_elms = soup.select('a[type="text/html"]')
        for a in a_elms:
            listData[i]['sub'].append({"sub_link": domain + a.get('href')})
#     print(listData)


        
# 建立 書本清單 的 json 檔 
def saveJson():
    fp = open("gutenberg.json", "w", encoding="utf-8")
    fp.write( json.dumps(listData, ensure_ascii=False, indent=4) )
    fp.close()
    

    
# 將所有書本內容，各自寫到 txt 與 json中
def writeTxt():
    fp = open("gutenberg.json", "r", encoding="utf-8")
    strJson = fp.read()
    fp.close()
    
 # 建立放置 txt 檔的資料夾
    folderPath = 'gutenberg_txt'
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
        
    listContent=[] #額外置放處理後的文章容器   
    
    listResult = json.loads(strJson)
    for i in range(5):
        for j in range(1):
            response = requests.get(listResult[i]['sub'][j]['sub_link'], headers = headers)

#         soup = BeautifulSoup(response.text, "lxml")
#         a_elms = soup.select('p[id]') #從html中獲得text > 不需要 因為response已經是整個網頁 
        #資料清洗   
#         for chres in a_elms: #article只包含中文內容 (過濾)
            regex02 = r'[\u4E00-\u9FFF，。：「」；、？！『』]+'
            string02 = response.text #針對整個網頁的內容去篩選正規化的結果
            match02 = re.findall(regex02, string02)
            Strtext = "".join(match02)
         
         

        # 決定 txt 的檔案名稱
            fileName = f"{listResult[i]['title']}.txt"
        
        # 將內容存到 txt 中
            fp = open(f"{folderPath}/{fileName}","w",encoding="utf-8")
            fp.write(Strtext)
            fp.close()
            
#         # 額外將書本內容放到 list 當中，建立 train.json
#             listContent.append(Strtext)
            
#         # 延伸之後的教學，建立訓練資料json 
#         fp = open("train_gutenberg.json","w",encoding="utf-8")
#         fp.write( json.dumps(listContent, ensure_ascii=False, indent=4) )
#         fp.close()

       
        

    
if __name__ == "__main__":
#     getLinks()
#     subLinks()
#     saveJson()
    writeTxt()